In [24]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc import psd
from simple_pe.param_est import find_metric_and_eigendirections
from simple_pe.param_est.pe import SimplePESamples
from simple_pe.waveforms import check_physical
from pesummary.utils.samples_dict import SamplesDict
plt.rcParams['text.usetex'] = False
import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)
from calcwf import *

We want to try and create a degeneracy line by starting at some quasicircular point and calculating the metric and resulting ellipse. We can then draw the smallest ellipse reaching some fiducial eccentricity value, and draw a line between these two points. We could also instead of fixing the fiducial eccentricity value use a fixed mismatch value. This may be more natural for scaling to other parameter values.

First let's write a function to find this point for a n-dimensional parameter space and metric object.

In [45]:
def find_extreme_point(metric, target_val, target_par, base_vals):

    # Points on an n-dimensional sphere
    n = len(metric.evec)
    points = np.random.rand(n, 10000)*2-1
    points /= np.linalg.norm(points, axis=0)

    # Transform to n-dimensional ellipsoid
    ellipse = np.matmul(metric.normalized_evecs().samples, points)
    keys = list(metric.dxs.keys())

    # Find point with most extreme target parameter
    if target_val < base_vals[target_par]:
        extreme_fn = np.argmin
    else:
        extreme_fn = np.argmax
        target_ind = keys.index(target_par)
    extreme_ind = extreme_fn(ellipse[target_ind])
    extreme_point = ellipse[:,extreme_ind]

    # Scale point to desired
    target_diff = target_val-base_vals[target_par]
    extreme_diff = extreme_point[target_ind]-base_vals[target_par]
    extreme_point *= target_diff/extreme_diff

    # Put point into dictionary format and add base vals
    extreme_vals = {}
    for i, key in enumerate(keys):
        extreme_vals[key] = extreme_point[i] + base_vals[key]

    return extreme_vals   

# 2D example (eccentricity, chirp mass)

First let's try eccentricity and chirp mass with a quasicircular chirp mass of 24 solar masses and a fiducial eccentricity value of 0.1. We can then compare the degeneracy line we compute from this method to the 'analytic' degeneracy line we have for this 2d parameter space.

In [6]:
# Defines PSD settings
ifos = ['H1']
psds = {'H1': 'aLIGOZeroDetHighPower',
        'f_low': 20,
        'f_high': 8192,
        'length': 32
        }
snr = 18
psds['delta_f'] = 1. / psds['length']
approximant = 'teobresums'

# Calculates PSD
pycbc_psd = {}
for ifo in ifos:
    pycbc_psd[ifo] = psd.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                psds['f_low'])
pycbc_psd['harm'] = 1. / sum([1. / pycbc_psd[ifo] for ifo in ifos])

# Uses simple-pe to calculate approx. of posterior dist. using metric, eigendirections
pars = {'chirp_mass':24, 'symmetric_mass_ratio':2/9, 'eccentricity': 0, 'spin_1z': 0, 'spin_2z': 0, 'f_ref':20}
par_dirs = ['eccentricity', 'chirp_mass']

g_e_chirp = find_metric_and_eigendirections(pars, par_dirs, snr=snr, f_low=psds['f_low'], psd=pycbc_psd['harm'], approximant=approximant)
g_e_chirp.calculate_evecs()

Calculating the metric | iteration 1 < 20 | error 0.00035 > 0.00036: 


In [77]:
# Generate match grid
e_chirp_mat = g_e_chirp.generate_match_grid(npts=21, scale=2)

In [ ]:
# Plot match grid
plt.tricontourf(e_chirp_mat.samples[1], e_chirp_mat.samples[0], e_chirp_mat['match'], levels=np.linspace(0.97, 1, 21))
plt.colorbar(label='match')

# Get extreme point and ellipse of metric
base_pars = {'eccentricity': 0, 'chirp_mass': 24}
extreme_point = find_extreme_point(g_e_chirp, 0.1, 'eccentricity', base_pars)
ellipse = g_e_chirp.generate_ellipse(npts=1000, scale=1.5)
plt.plot(ellipse['chirp_mass'], ellipse['eccentricity'], c='orange')
plt.plot([base_pars['chirp_mass'], extreme_point['chirp_mass']], [base_pars['eccentricity'], extreme_point['eccentricity']], c='magenta', label='metric')

# Plot 'analytic' degeneracy line
e_vals = np.linspace(0,0.1,101)
chirp_vals = chirp_degeneracy_line(base_pars['chirp_mass'], e_vals, sample_rate=4096, f_low=20, q=2, f_match=20, return_delta_m=False)
plt.plot(chirp_vals, e_vals, c='k', label='analytic')
plt.xlabel('chirp')
plt.ylabel('e')
plt.ylim(0,)
plt.legend()